In [27]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing as bwp
from utilities import *

In [13]:
bd.projects

Brightway2 projects manager with 10 objects:
	LCA_VL_masterdb
	Test
	as23_project
	as23_project_bw2
	default
	ei39
	ei39_premise_scenarios
	premise_ei39
	tictac
	🪒
Use `projects.report()` to get a report on all projects.

In [18]:
bd.projects.set_current("ei39")

In [19]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent_cutoff_3.9_remind_SSP2-PkBudg1150_2020
	ei39

In [31]:
del bd.databases['ecoinvent_cutoff_3.9_remind_SSP2-PkBudg1150_2020']

Vacuuming database 


In [22]:
from fs.zipfs import ZipFS

In [26]:
premise_dp = bwp.load_datapackage(ZipFS('8420384.zip'))

In [49]:
premise_dp.metadata

{'profile': 'tabular-data-package',
 'resources': [{'path': 'data/inventories.csv',
   'profile': 'tabular-data-resource',
   'name': 'inventories',
   'format': 'csv',
   'mediatype': 'text/csv',
   'encoding': 'utf-8',
   'schema': {'fields': [{'name': 'Database',
      'type': 'string',
      'format': 'default'},
     {'name': 'remind SSP2-PkBudg1150',
      'type': 'string',
      'format': 'default'}],
    'missingValues': ['']}},
  {'path': 'data/scenario_data.csv',
   'profile': 'tabular-data-resource',
   'name': 'scenario_data',
   'format': 'csv',
   'mediatype': 'text/csv',
   'encoding': 'utf-8',
   'schema': {'fields': [{'name': 'from activity name',
      'type': 'string',
      'format': 'default'},
     {'name': 'from reference product', 'type': 'string', 'format': 'default'},
     {'name': 'from location', 'type': 'string', 'format': 'default'},
     {'name': 'from categories', 'type': 'string', 'format': 'default'},
     {'name': 'from database', 'type': 'string', 'f

In [50]:
bc.LCA??

Init signature:
bc.LCA(
    demand: dict,
    method: Optional[tuple] = None,
    weighting: Optional[str] = None,
    normalization: Optional[str] = None,
    data_objs: Optional[Iterable[Union[pathlib.Path, fs.base.FS, bw_processing.datapackage.DatapackageBase]]] = None,
    remapping_dicts: Optional[Iterable[dict]] = None,
    log_config: Optional[dict] = None,
    seed_override: Optional[int] = None,
    use_arrays: bool = False,
    use_distributions: bool = False,
)
Source:        
class LCA(Iterator):
    """An LCI or LCIA calculation.

    Compatible with Brightway2 and 2.5 semantics. Can be static, stochastic, or iterative (scenario-based), depending on the ``data_objs`` input data..

    """
    matrix_labels = [
        "technosphere_mm",
        "biosphere_mm",
        "characterization_mm",
        "normalization_mm",
        "weighting_mm",
    ]

    #############
    ### Setup ###
    #############

    def __init__(
        self,
        demand: dict,
        # Brightw

In [33]:
bd.projects.migrate_project_25()

Updating all LCIA methods


762it [03:19,  3.81it/s]


Updating all LCI databases


2it [00:37, 18.94s/it]


In [40]:
pear_market = bd.get_node(name="market for pear", database="ei39")
pear_china = bd.get_node(name="pear production", location="CN", database="ei39")
apple = bd.get_node(name="apple production", location="CL", database="ei39")

In [41]:
ipcc = ('IPCC 2021', 'climate change: fossil', 'global warming potential (GWP100)')
demand = {pear_market: 1}

In [42]:
# Get the list of `data_objs` - our new datapackage will be appended to this list.
fu, data_objs, remapping = bd.prepare_lca_inputs(demand=demand, method=ipcc)
lca = bc.LCA(fu, data_objs=data_objs, remapping_dicts=remapping)
lca.lci()
lca.lcia()
print("Pear:", lca.score)


Pear: 0.446075994204733


In [43]:
dp = safety_razor(
    consumer=pear_market,
    previous_producer=pear_china, 
    new_producer=apple, 
)
lca = bc.LCA(fu, data_objs=data_objs + [premise_dp, dp], remapping_dicts=remapping)
lca.lci()
lca.lcia()
print("Apples are not pears:", lca.score)

KeyError: 'combinatorial'